In [2]:
import pandas as pd
import time

In [8]:
d = pd.read_csv("data/training_data.csv.gz", compression="gzip",
               dtype={"age_gr": str, "ev_code": str, "mtype_sec": str, "event_id": str})

In [9]:
print("transactions: {}, unique customers: {}".format(len(d), d.cust_id.nunique()))

transactions: 18609023, unique customers: 5684182


In [10]:
d.head()

,cust_id,name,surname,dob_y,dob_m,dob_d,acc_y,mj_tnum,perf_y,perf_m,...,purch_m,paid_tix,free_tix,paid,event_id,mtype_sec,ev_code,age_tp,age_ta,age_gr
0,10995205,peter,coumbiadis,1974,9,2,2009,8813,2015,8,...,8,2,0.0,54.40,281282,LEAGUE,EOKI0000015D,41,35,40-44
1,12663067,emily,payne,1980,8,16,2010,40865,2015,4,...,4,1,0.0,31.50,279088,SOCCER,EMVC2015706,35,30,35-39
2,13852676,renee,blandin de chalain,1989,5,10,2011,92757,2014,9,...,9,3,0.0,140.50,279738,AFL,ESSW0000014QF,25,22,25-29
3,3735954,melanie,pring,1990,4,9,2003,25650,2014,9,...,9,2,0.0,120.35,279738,AFL,ESSW0000014QF,24,13,18-24
4,16576858,trish,simunovic,1971,11,28,2013,84,2014,10,...,10,4,0.0,109.95,281309,SOCCER,ECMR2014906,43,42,40-44


In [11]:
d1 = pd.get_dummies(d[["mtype_sec","age_gr"]], columns=["mtype_sec"])
d1.head()

,age_gr,mtype_sec_5060S,mtype_sec_70SPOP,mtype_sec_70SROCK,mtype_sec_80SPOP,mtype_sec_80SROCK,mtype_sec_90SPOP,mtype_sec_90SROCK,mtype_sec_AFL,mtype_sec_ALTERN,...,mtype_sec_RNB,mtype_sec_ROCK,mtype_sec_RUNION,mtype_sec_SCHOOL,mtype_sec_SENIOR,mtype_sec_SFCOMEDY,mtype_sec_SFMUSIC,mtype_sec_SOCCER,mtype_sec_STANDUPC,mtype_sec_TENNIS
0,40-44,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,35-39,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,25-29,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,18-24,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,40-44,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [12]:
d1.shape

(18609023, 46)

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.preprocessing import LabelEncoder

In [14]:
# le = LabelEncoder()

In [15]:
d1["age_gr"].unique()

array(['40-44', '35-39', '25-29', '18-24', '30-34', '60-64', '45-49',
       '50-54', '55-59', '00-17', '75-84', '70-74', '65-69'], dtype=object)

In [16]:
# le.fit(d1["age_gr"])

In [17]:
# le.classes_

In [18]:
X_train, X_test, y_train, y_test = train_test_split(d1.drop("age_gr", axis=1), d1["age_gr"], 
                                                    test_size=0.3, random_state=94, stratify=d1["age_gr"])

In [19]:
print("created training data - features {} and target {}".format(X_train.shape, y_train.shape))

created training data - features (13026316, 45) and target (13026316,)


In [23]:
gnb = BernoulliNB()
gnb.fit(X_train, y_train)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

svc = SVC(kernel='linear')
svc.fit(X_train, y_train)

print("accuracy: {:.3f}".format(accuracy_score(y_test, svc.predict(X_test))))
scores = cross_val_score(svc, X_train, y_train, cv=5)


#print("accuracy: {:.3f}".format(accuracy_score(y_test, gnb.predict(X_test))))
# scores = cross_val_score(gnb, X_train, y_train, cv=5)
print("cross-validation scores: {}, {}, {}, {}, {}".format(*scores))

In [11]:


# t0 = time.time()
# d1 = d[["mj_tnum","mtype_sec","paid_tix"]].groupby(["mj_tnum","mtype_sec"]).agg(["count","sum"])
# d1.columns = ["_".join(col_tpl) for col_tpl in d1.columns.values]
# d1["paid_for"] = pd.Series([tpl[1] for tpl in d1.index.values], index=d1.index, dtype=str)
# d1 = pd.get_dummies(d1)
# d1.index = d1.index.droplevel(1)
# d1 = d1.drop(["paid_tix_count","paid_tix_sum"], axis=1)
# t1 = time.time()
# print("elapsed time: {:.0f} m {:.0f} s".format(*divmod(t1-t0,60)))

In [11]:
g = pd.read_csv("data/attendances_over_time.csv.gz", encoding='latin-1', 
                header=None, names=["cust_id","age_gr","mtype_sec", "n_attend"])

In [12]:
g.head()

,cust_id,age_gr,mtype_sec,n_attend
0,11168,50-54,RUNION,1
1,11171,50-54,RUNION,1
2,11179,60-64,FOLK,1
3,11179,60-64,AFL,5
4,11181,55-59,RUNION,24


In [15]:
from collections import defaultdict

f = defaultdict(lambda: defaultdict(int))

for k in g.groupby(["cust_id", "age_gr"]).groups:   # k is a key of the form (11179, '60-64')
    f[k[0]].update({"$"+mtype: 1 for mtype in g.ix[f[k],"mtype_sec"].tolist()})

{(11168, '50-54'): Int64Index([0], dtype='int64'),
 (11171, '50-54'): Int64Index([1], dtype='int64'),
 (11179, '60-64'): Int64Index([2, 3], dtype='int64'),
 (11181, '55-59'): Int64Index([4], dtype='int64'),
 (11196, '30-34'): Int64Index([6], dtype='int64'),
 (11196, '35-39'): Int64Index([5], dtype='int64'),
 (11197, '50-54'): Int64Index([7], dtype='int64'),
 (11206, '50-54'): Int64Index([8], dtype='int64'),
 (11228, '40-44'): Int64Index([9, 10], dtype='int64'),
 (11249, '50-54'): Int64Index([11], dtype='int64'),
 (11277, '50-54'): Int64Index([12, 13, 14], dtype='int64'),
 (11284, '40-44'): Int64Index([15], dtype='int64'),
 (11285, '45-49'): Int64Index([16, 17, 18, 19, 20, 21], dtype='int64'),
 (11302, '40-44'): Int64Index([22], dtype='int64'),
 (11322, '60-64'): Int64Index([23], dtype='int64'),
 (11344, '55-59'): Int64Index([24], dtype='int64'),
 (11357, '55-59'): Int64Index([25], dtype='int64'),
 (11369, '60-64'): Int64Index([26], dtype='int64'),
 (11371, '40-44'): Int64Index([27], dt